## Orienting waffle images with bright star cause

Craig Lage 25-Sep-23


In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
import lsst.geom as geom
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient
from lsst.geom import Box2I, Point2I, SpherePoint
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

In [ ]:
butler = Butler('/repo/LATISS', collections="LATISS/runs/quickLook")

In [ ]:
# Now get the image data and the metadata

waffles = [[2023052300612, "HD143275", 240.08330, -22.622], [2023071800284, "M8", 270.904, -24.378], \
          [2023092100484, "HD216956", 344.415, -29.6235], [2023092100502, "HD214952", 340.6682, -46.8844]]
expId = 2023052300612
objectRA = Angle(240.08330 * u.deg)
objectDec = Angle(-22.622 * u.deg)

expId = 2023071800284
objectRA = Angle(270.904 * u.deg)
objectDec = Angle(-24.378 * u.deg)

#expId = 2023092100484
#objectRA = Angle(344.415 * u.deg)
#objectDec = Angle(-29.6235 * u.deg)

#expId = 2023092100502
#objectRA = Angle(340.6682 * u.deg)
#objectDec = Angle(-46.8844 * u.deg)



In [ ]:
waffles = [[2023052300612, "HD143275", 240.08330, -22.622], [2023071800284, "M8", 270.904, -24.378], \
          [2023092100484, "HD216956", 344.415, -29.6235], [2023092100502, "HD214952", 340.6682, -46.8844]]
waffles = [[2023052300612, "HD142990", 239.6452, -24.8317], [2023071800284, "M8", 270.904, -24.378], \
          [2023092100484, "HD216956", 344.415, -29.6235], [2023092100502, "HD214952", 340.6682, -46.8844]]
waffles = [[2023092100479, "HD216956", 344.415, -29.6235]]
waffles = [[2023092100475, "HD216956", 344.415, -29.6235]]
waffles = [[2023092100525, "HD216956", 344.41269272, -29.62223703]] # ICRS from simbad
for [expId, objectID, RA, Dec] in waffles:
    exp = butler.get('quickLookExp', detector=0, exposure=expId)
    mData = exp.getMetadata()
    date_beg = Time(mData['DATE-BEG'], format='isot', scale='tai')
    time = date_beg.utc
    print(expId, time)
    
    objectRA = Angle(RA * u.deg)
    objectDec = Angle(Dec * u.deg)

    # Used for arrow locations
    Ncenter = (700.0, 900.0)
    Nlength = 500.0
    NcenterAzEl = (3200, 700)
    NcenterPA = (3000, 2000)
    Nlabel = 650.0
    yShift = 150.0
    NcenterFW = (3200, 3000)
    FW_shift = 100
    labelShift = 40

    el = Angle(mData['ELSTART'] * u.deg)
    az = Angle(mData['AZSTART'] * u.deg)
    dec = Angle(mData['DECSTART'] * u.deg)

    plt.figure(figsize=(16,16))
    arr = exp.image.array
    arr = np.clip(arr, 1, 100000) # This image has some negative values, and this removes them
    img = plt.imshow(arr, norm=LogNorm(vmin=1000, vmax=100000),  interpolation='Nearest', cmap='gray')

    # Get Az, El shift from wcs:
    wcs = exp.getWcs()
    origin = wcs.getSkyOrigin()
    vi = exp.getInfo().getVisitInfo()

    skyLocation = SkyCoord(origin.getRa().asRadians(), origin.getDec().asRadians(), unit=u.rad)
    altAz = AltAz(obstime=vi.date.toPython(), location=AUXTEL_LOCATION)
    obsAltAz = skyLocation.transform_to(altAz)
    shiftInArcsec = 50.0
    altShifted = SkyCoord(obsAltAz.az, obsAltAz.alt + Angle(shiftInArcsec * u.arcsec), frame=altAz)
    altShiftedSpherePoint = SpherePoint(altShifted.icrs.ra.deg*geom.degrees, altShifted.icrs.dec.deg*geom.degrees)
    azShifted = SkyCoord(obsAltAz.az + Angle(shiftInArcsec / np.cos(obsAltAz.alt) * u.arcsec), obsAltAz.alt, frame=altAz)
    azShiftedSpherePoint = SpherePoint(azShifted.icrs.ra.deg*geom.degrees, azShifted.icrs.dec.deg*geom.degrees)
    originPixel = wcs.skyToPixel(origin)
    altShiftedPixel = wcs.skyToPixel(altShiftedSpherePoint)
    altShift = altShiftedPixel - originPixel
    azShiftedPixel = wcs.skyToPixel(azShiftedSpherePoint)
    azShift = azShiftedPixel - originPixel

    # Now plot the Az El arrows as determined by wcs
    plt.arrow(originPixel.x, originPixel.y, azShift.x, azShift.y,\
        color='orange', width = 20)
    plt.text(azShiftedPixel.x + labelShift, azShiftedPixel.y + labelShift  , 'AZ - WCS', fontsize=12, weight='bold', color='orange')
    plt.arrow(originPixel.x, originPixel.y, altShift.x, altShift.y,\
        color='orange', width = 20)    
    plt.text(altShiftedPixel.x + labelShift  , altShiftedPixel.y + labelShift  , 'EL - WCS', fontsize=12, weight='bold', color='orange')


    # Get N, E shift from wcs:
    decShifted = SkyCoord(skyLocation.ra, skyLocation.dec + Angle(shiftInArcsec * u.arcsec))
    decShiftedSpherePoint = SpherePoint(decShifted.ra.deg*geom.degrees, decShifted.dec.deg*geom.degrees)
    raShifted = SkyCoord(skyLocation.ra + Angle((shiftInArcsec / np.cos(skyLocation.dec)) * u.arcsec), skyLocation.dec)
    raShiftedSpherePoint = SpherePoint(raShifted.ra.deg*geom.degrees, raShifted.dec.deg*geom.degrees)
    originPixel = wcs.skyToPixel(origin)
    decShiftedPixel = wcs.skyToPixel(decShiftedSpherePoint)
    decShift = decShiftedPixel - originPixel
    raShiftedPixel = wcs.skyToPixel(raShiftedSpherePoint)
    raShift = raShiftedPixel - originPixel

    # Now plot the N E arrows as determined by wcs
    plt.arrow(originPixel.x, originPixel.y, decShift.x, decShift.y,\
        color='magenta', width = 20)
    plt.text(decShiftedPixel.x + labelShift, decShiftedPixel.y + labelShift  , 'N - WCS', fontsize=12, weight='bold', color='magenta')
    plt.arrow(originPixel.x, originPixel.y, raShift.x, raShift.y,\
       color='magenta', width = 20)    
    plt.text(raShiftedPixel.x + labelShift  , raShiftedPixel.y + labelShift  , 'E - WCS', fontsize=12, weight='bold', color='magenta')

    # Now plot the N, E arrows as determined by ROTPA
    rotpa = Angle(mData['ROTPA']*u.deg)
    plt.arrow(Ncenter[0], Ncenter[1], -Nlength*np.sin(rotpa.rad), Nlength*np.cos(rotpa.rad),\
              color='lightgreen', width = 20)
    plt.text(Ncenter[0]-Nlabel*np.sin(rotpa),Ncenter[1]+Nlabel*np.cos(rotpa), 'N', \
        color='lightgreen', fontsize=12, weight='bold')
    plt.arrow(Ncenter[0],Ncenter[1], Nlength*np.cos(rotpa.rad), Nlength*np.sin(rotpa.rad),\
        color='lightgreen', width = 20)
    plt.text(Ncenter[0]+Nlabel*np.cos(rotpa),Ncenter[1]+Nlabel*np.sin(rotpa), 'E', \
        color='lightgreen', fontsize=12, weight='bold')

    # Now calculate and plot the Az, El arrows from my algorithm
    sinTheta = max(-1.0, min(1.0,float(np.cos(AUXTEL_LOCATION.lat) / np.cos(dec) * np.sin(az))))
    cosTheta = (np.sin(el) * np.sin(dec) - np.sin(AUXTEL_LOCATION.lat)) / (np.cos(el) * np.cos(dec))
    #print(sinTheta, cosTheta)
    theta = Angle(np.arcsin(sinTheta) * u.rad)
    if cosTheta > 0:
        rotAzEl = rotpa - theta
    else:    
        rotAzEl = rotpa - (Angle(180.0 * u.deg) - theta)


    plt.arrow(NcenterAzEl[0],NcenterAzEl[1], Nlength*np.sin(rotAzEl.rad),-Nlength*np.cos(rotAzEl.rad),\
        color='cyan', width = 20)
    plt.text(NcenterAzEl[0] + Nlabel*np.sin(rotAzEl),NcenterAzEl[1] - Nlabel*np.cos(rotAzEl), 'EL', \
        color='cyan', fontsize=12, weight='bold')
    plt.arrow(NcenterAzEl[0],NcenterAzEl[1], Nlength*np.cos(rotAzEl.rad), Nlength*np.sin(rotAzEl.rad),\
        color='cyan', width = 20)
    plt.text(NcenterAzEl[0]+Nlabel*np.cos(rotAzEl),NcenterAzEl[1]+Nlabel*np.sin(rotAzEl), 'AZ', \
        color='cyan', fontsize=12, weight='bold')


    #print(az.deg, el.deg)
    #print(theta.deg, rotpa.deg, rotAzEl.deg)

    object = SkyCoord(objectRA, objectDec)
    objectSpherePoint = SpherePoint(object.icrs.ra.deg*geom.degrees, object.icrs.dec.deg*geom.degrees)
    objectPixels = wcs.skyToPixel(objectSpherePoint)
    plt.text(objectPixels.x, objectPixels.y, "X", fontsize=48, color='red')
    print(objectPixels.x, objectPixels.y, \
          f"Total distance = {np.sqrt(objectPixels.x**2+objectPixels.y**2)/10.0/3600.0} degrees")
    #plt.xlim(-10000, 5000)
    #plt.ylim(-10000, 5000)
    plt.savefig(f'/home/craiglagegit/DATA/Waffle_{expId}.png')

In [ ]:
rotpa.rad

In [ ]:
print(origin.getRa().asDegrees(), origin.getDec().asDegrees())

In [ ]:
objectRA = Angle(240.08330 * u.deg)
objectDec = Angle(-22.622 * u.deg)

In [ ]:
object = SkyCoord(objectRA, objectDec)

In [ ]:
objectSpherePoint = SpherePoint(object.icrs.ra.deg*geom.degrees, object.icrs.dec.deg*geom.degrees)

In [ ]:
objectPixels = wcs.skyToPixel(objectSpherePoint)

In [ ]:
objectPixels.y

In [ ]:
sinTheta =  np.cos(AUXTEL_LOCATION.lat) / np.cos(dec) * np.sin(az)

In [ ]:
sinTheta